# Text Generation using RNNs with TensorFlow

In [17]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding

In [63]:
tf.version.VERSION

'2.16.0-rc0'

In [3]:
path_to_file = 'bible.txt'

text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8') 
print('Length of text: {} characters'.format(len(text)))

Length of text: 4432261 characters


In [33]:
vocab = sorted(set(text))

In [30]:
print(text[:500])

1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day.

1:6 An


For preparation of the data, It is ideal to vectorize the words into characters, so the model is trained to predict the next character in order to create words.

In [10]:
sample_texts = ['Su', 'absentaso', '?']

chars = tf.strings.unicode_split(sample_texts, input_encoding = 'UTF-8')
chars

<tf.RaggedTensor [[b'S', b'u'], [b'a', b'b', b's', b'e', b'n', b't', b'a', b's', b'o'],
 [b'?']]>

The previous code define a byte for every different characters, so now using the previosly defined vocavulary, I can transform that into integers

In [14]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary = list(vocab), mask_token = None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[43, 70], [50, 51, 68, 54, 63, 69, 50, 68, 64], [24]]>

And the same can be done to get the characters from integers, so I can train the model with sequence of numbers instead of characters, and then decoed it

In [15]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary = ids_from_chars.get_vocabulary(), invert = True, mask_token = None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'S', b'u'], [b'a', b'b', b's', b'e', b'n', b't', b'a', b's', b'o'],
 [b'?']]>

In [16]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'Su', b'absentaso', b'?'], dtype=object)

In [21]:
def ids_to_text(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [19]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) 

In [23]:
def split_into_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset_xy = sequences.map(split_into_input_target)

for input_example, target_example in dataset_xy.take(1):
    print("Input :", ids_to_text(input_example).numpy())
    print("Target:", ids_to_text(target_example).numpy())

Input : b'1:1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, an'
Target: b':1 In the beginning God created the heaven and the earth.\r\n\r\n1:2 And the earth was without form, and'


In [39]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset_xy
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [51]:
vocab_size = len(vocab)

## Our embedding dimension
embedding_dim = 256

## number of RNN units
rnn_units = 1024

In [69]:
model = Sequential([
    Embedding(vocab_size,
            embedding_dim),
    GRU(rnn_units,
        return_sequences=True,
        stateful=True),
    Dense(vocab_size)
])
model.compile('rmsprop', 'mse')

In [71]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)